In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# path of dataset file
import os
dataset = "/content/drive/MyDrive/Lung Dataset /Chest CT Scan Images/"
print(os.listdir(dataset))


['large.cell.carcinoma', 'adenocarcinoma', 'squamous.cell.carcinoma', 'normal']


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from PIL import Image
from torchvision import transforms
from PIL import Image

import random
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# Define the LeNet architecture
class LeNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(LeNetFeatureExtractor, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # Adjusted this line
        self.fc2 = nn.Linear(120, 84)

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 16 * 5 * 5)  # Adjusted this line
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

# Instantiate the model
lenet_extractor = LeNetFeatureExtractor()
lenet_extractor.eval()  # Set the model to evaluation mode

# Define the transform to match LeNet's input requirements
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((32, 32)),                 # Resize to 32x32
    transforms.ToTensor()                        # Convert to tensor
])


In [ ]:
# Assuming `dataset` is the path to your dataset
classes = os.listdir(dataset)
features_list = []
labels_list = []

for label, class_dir in enumerate(classes):
    class_path = os.path.join(dataset, class_dir)
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        image = Image.open(image_path).convert('L')  # Open image and convert to grayscale
        image = transform(image)                    # Apply transforms
        image = image.unsqueeze(0)                  # Add batch dimension

        with torch.no_grad():
            features = lenet_extractor(image)       # Extract features
            features = features.numpy().flatten()   # Flatten the output to a 1D array

        features_list.append(features)
        labels_list.append(label)

features = np.vstack(features_list)
labels = np.hstack(labels_list)

print(f'Features shape: {features.shape}')
print(f'Labels shape: {labels.shape}')


Features shape: (1000, 84)
Labels shape: (1000,)


In [ ]:

# Split data
X_train, X_test, y_train, y_test = train_test_split(features, labels_list, test_size=0.3, random_state=42)



In [ ]:

# Assuming X_train, X_test, y_train, y_test are already defined

# Function to plot ROC curve
def plot_roc_curve(fpr, tpr, roc_auc, title):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

# Train and evaluate SVM
svm = SVC(kernel='poly', probability=True)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
y_score_svm = svm.decision_function(X_test)

# Compute metrics for SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
fpr_svm, tpr_svm, _ = roc_curve(y_test, y_score_svm)
roc_auc_svm = auc(fpr_svm, tpr_svm)

print(f'SVM Accuracy: {accuracy_svm:.4f}')
print(f'SVM Precision: {precision_svm:.4f}')
print(f'SVM Recall: {recall_svm:.4f}')
print(f'SVM F1 Score: {f1_svm:.4f}')
print(f'SVM AUC: {roc_auc_svm:.4f}')

# Plot ROC curve for SVM
plot_roc_curve(fpr_svm, tpr_svm, roc_auc_svm, 'SVM ROC Curve')

# Confusion matrix for SVM
cm_svm = confusion_matrix(y_test, y_pred_svm)
disp_svm = ConfusionMatrixDisplay(confusion_matrix=cm_svm)
disp_svm.plot()
plt.title('SVM Confusion Matrix')
plt.show()



ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].